## --- [Day 14: Docking Data](https://adventofcode.com/2020/day/14) ---

In [1]:
import re

INPUT_FILE = 'input_d14.txt'
EXAMPLE_FILE = 'input2_d14.txt'

class Bitmask:
    def __init__(self, mask: str):
        '''
        Initialize from a string as described in the puzzle:
        A string of 36 bits with the most significant bit on the left.
        When the value is 'X' that bit is ignored.
        :param mask: the initial mask value
        '''
        self.ones = None
        self.zeroes = None
        self.xes = None
        self.mask_str = None
        self.update(mask)
        
    def update(self, mask: str):
        self.mask_str = mask
        self.ones = 0
        self.zeroes = 0
        self.xes = 0
        
        # Process each "bit" (1, 0, X) left to right
        for b in mask:
            self.ones <<= 1
            self.zeroes <<= 1
            self.xes <<= 1
            
            if b == '1':
                self.ones += 1
                self.zeroes += 1
                self.xes += 1
            elif b == '0':
                self.xes += 1
            elif b == 'X':
                self.zeroes += 1
    
    def apply(self, input_val: int) -> int:
        '''
        Applies the current mask to the input value
        :param input_val: value to be masked
        :return: the masked value
        '''
        return input_val & self.zeroes | self.ones
    
    def print_mask(self):
        '''
        Prints the current mask representation
        '''
        print(f'IN:      {self.mask_str}')
        print(f'OR 1\'s:  {self.ones:036b}')
        print(f'AND 0\'s: {self.zeroes:036b}')
        print(f'   X\'s:  {self.xes:036b}')

In [2]:
class Decoder:
    '''
    Reads and processes input program as defined in puzzle
    '''
    
    def __init__(self, input_file: str=INPUT_FILE):
        '''
        :param input_file: Input file containing the program
        '''
        self.memory = {}
        self.mask = None
        self.input_file = input_file
        
    def handle_command(self, tokens: dict):
        '''
        Given a dict representing a single command, updates the
        state accordingly. The dict contains 3 keys:
        * 'cmd': either 'mask' or 'mem'
        * 'addr': memory address (unused when cmd is mask)
        * 'val': the value to be assigned
        :param tokens: dict as described above
        '''
        if tokens['cmd']=='mask':
            self.mask = Bitmask(tokens['val'])
        elif tokens['cmd'] == 'mem':
            masked_val = self.mask.apply(int(tokens['val']))
            self.memory[int(tokens['addr'])] = masked_val
        else:
            raise IllegalArgument
    
    def run(self):
        '''
        Loads the input file and processes the commands
        '''
        with open(self.input_file) as fh:
            for line in fh.readlines():
                pattern = r"(?P<cmd>mask|mem)\[?(?P<addr>\d*)\]? = (?P<val>.*)"
                tokens = re.match(pattern, line).groupdict()
                self.handle_command(tokens)

In [3]:
# Bitmask tests
mask = Bitmask('XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X')
assert mask.ones == 64
assert mask.apply(11) == 73
assert mask.zeroes == 68719476733

In [4]:
# Test against example input
ex1 = Decoder(EXAMPLE_FILE)
ex1.run()
assert ex1.memory == {8:64, 7:101}
assert sum(ex1.memory.values()) == 165

In [5]:
# Part 1 solution
part1 = Decoder(INPUT_FILE)
part1.run()
sum(part1.memory.values())

14553106347726

### Part 2

    If the bitmask bit is 0, the corresponding memory address bit is unchanged.
    If the bitmask bit is 1, the corresponding memory address bit is overwritten with 1.
    If the bitmask bit is X, the corresponding memory address bit is floating.


In [6]:
from typing import Set

class Floatmask(Bitmask):
    def __init__(self, mask: str):
        self.update(mask)
        
    def update(self, mask: str):
        super().update(mask)
        
        # Set the X bits as 0's
        for i in range(len(self.mask_str)):
            if self.mask_str[i] == 'X':
                self.zeroes &= ~(1 << (len(self.mask_str)-i-1))

        self.floaters = []
        for i in range(len(self.mask_str)):
            if self.mask_str[i] == 'X':
                self.floaters.append(1 << (len(self.mask_str)-i-1))

    def apply(self, value: int) -> Set[str]:
        '''
        Applies the current 1/0/X mask to the value
        :return: set of all masked values
        '''
        values = set()
        
        # First append the value with the 1/0 mask applied
        values.add(value & self.xes | self.ones)
        
        # Now create every variant based on the Xes in the string
        for fval in self.floaters:
            extend_by = []
            for val in values:
                extend_by.append(val+fval)
            values.update(extend_by)
        
        return values
        
        
        

In [7]:
class FloatDecoder(Decoder):

    def handle_command(self, tokens: dict):
        '''
        Updated for part 2: applies Floatmask to memory address and
        writes to all possible addresses
        Given a dict representing a single command, updates the
        state accordingly. The dict contains 3 keys:
        * 'cmd': either 'mask' or 'mem'
        * 'addr': memory address (unused when cmd is mask)
        * 'val': the value to be assigned
        :param tokens: dict as described above
        '''

        if tokens['cmd']=='mask':
            self.mask = Floatmask(tokens['val'])
        elif tokens['cmd'] == 'mem':
            addresses = self.mask.apply(int(tokens['addr']))
            for address in addresses:
                self.memory[address] = int(tokens['val'])
        else:
            raise IllegalArgument



In [8]:
# Floatmask tests
fmask = Floatmask('000000000000000000000000000000X1001X')
assert fmask.floaters == [32,1]
assert fmask.apply(42) == {26,27,58,59}

In [9]:
# Example input test
ex2 = FloatDecoder('input3_d14.txt')
ex2.run()
assert sum(ex2.memory.values()) == 208

In [10]:
# Part 2 solution
part2 = FloatDecoder(INPUT_FILE)
part2.run()
sum(part2.memory.values())

2737766154126